<a href="https://colab.research.google.com/github/aaalexlit/wandb_notebooks/blob/main/W%26B_02_Split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install wandb -qq
!wget -nc https://raw.githubusercontent.com/wandb/edu/main/mlops-001/lesson1/params.py

In [2]:
import os, warnings
import wandb

import pandas as pd
from fastai.vision.all import *
from sklearn.model_selection import StratifiedGroupKFold

import params
warnings.filterwarnings('ignore')

In [4]:
run = wandb.init(project=params.WANDB_PROJECT,
                 entity=params.ENTITY,
                 job_type="data_split")

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
raw_data_at = run.use_artifact(f'{params.RAW_DATA_AT}:latest')
path = Path(raw_data_at.download())

wandb: Downloading large artifact bdd_simple_1k:latest, 813.77MB. 4007 files... 
wandb:   4007 of 4007 files downloaded.  
Done. 0:1:29.7


In [6]:
path.ls()

(#5) [Path('artifacts/bdd_simple_1k:v0/images'),Path('artifacts/bdd_simple_1k:v0/eda_table.table.json'),Path('artifacts/bdd_simple_1k:v0/LICENSE.txt'),Path('artifacts/bdd_simple_1k:v0/labels'),Path('artifacts/bdd_simple_1k:v0/media')]

In [7]:
orig_eda_table = raw_data_at.get("eda_table")

wandb: Downloading large artifact bdd_simple_1k:latest, 813.77MB. 4007 files... 
wandb:   4007 of 4007 files downloaded.  
Done. 0:0:3.8


In [8]:
fnames = orig_eda_table.get_column("File_Name")
groups = [s.split('-')[0] for s in fnames]
y = orig_eda_table.get_column('bicycle')

In [9]:
df = pd.DataFrame()
df['File_Name'] = fnames
df['fold'] = -1

In [14]:
cv = StratifiedGroupKFold(n_splits=10)
for i, (train_idxs, test_idxs) in enumerate(cv.split(fnames, y, groups)):
  df.loc[test_idxs, ['fold']] = i

In [16]:
df['Stage'] = 'train'
df.loc[df.fold == 0, ['Stage']] = 'test'
df.loc[df.fold == 1, ['Stage']] = 'valid'
del df['fold']
df.Stage.value_counts()

train    800
test     100
valid    100
Name: Stage, dtype: int64

In [17]:
df.to_csv('data_split.csv', index=False)

In [19]:
processed_data_at = wandb.Artifact(params.PROCESSED_DATA_AT, type='split_data')

In [20]:
processed_data_at.add_file('data_split.csv')
processed_data_at.add_dir(path)

wandb: Adding directory to artifact (./artifacts/bdd_simple_1k:v0)... Done. 11.5s


In [21]:
data_split_table = wandb.Table(dataframe=df[['File_Name', 'Stage']])

In [22]:
join_table = wandb.JoinedTable(orig_eda_table, data_split_table, 'File_Name')

In [23]:
processed_data_at.add(join_table, 'eda_table_data_split')

ArtifactManifestEntry(path='eda_table_data_split.joined-table.json', digest='WYcWVGdz2OBIcBgr36X7QA==', ref=None, birth_artifact_id=None, size=127, extra={}, local_path='/root/.local/share/wandb/artifacts/staging/tmpg1_zxci4')

In [24]:
run.log_artifact(processed_data_at)
run.finish()